In [110]:
import re 
import numpy as np
import pandas as pd 
import matplotlib as mp 


Danish CPR numbers consist of a six-digit date (DDMMYY), followed by a 4 digit identifier (IIII). Using the re module, the task is to write and apply a regular expression that matches any CPR number, either in the DDMMYYIIII format, or in the DDMMYY-IIII format. The regular expression should contain four groups, such that the DD, MM, YY, and IIII parts can be extracted after matching.


In [111]:
text = '''adakaf
310702-7121
dad, 1001011110,fksfjwokff., 31314
222276-4390
1315208121
	'''
#A pattern that matches CPR in DDMMYYIIII format
# 
pattern= re.compile(r'(\d\d)(\d\d)(\d\d)-?(\d{4})')


In [112]:
match= pattern.finditer(text)
for x in match : 
        print(x.groups())
        dd = x.group(1)
        mm= x.group(2)
        yy =x.group(3)
        zz= x.group(4)
        print (yy)
        if '0'<=yy<='99' and '1'<=zz<='3999' :
            print ('Born in 1900')
    
        elif  '0'<=yy<='36' and '4000'<=zz<='4999' :
            print ('Born in 2000')
    
        elif  '37'<=yy<='99' and '4000'<=zz<='4999' :
            print ('Born in 1900')
    
        elif  '00'<=yy<='57' and '5000'<=zz<='8999' :
            print ('Born in 2000')
    
        elif  '58'<=yy<='99' and '5000'<=zz<='8999' :
            print ('Born in 1800')
        
        elif  '00'<yy<'36' and '9000'<zz<'9999' :
            print ('Born in 2000')
    
        elif  '37'<yy<'99' and '9000'<zz<'9999' :
            print ('Born in 1900')

('31', '07', '02', '7121')
02
Born in 2000
('10', '01', '01', '1110')
01
Born in 1900
('22', '22', '76', '4390')
76
Born in 1900
('13', '15', '20', '8121')
20
Born in 2000


Processing the FakeNewsCorpus data set

In [163]:
df = pd.read_csv('news_sample.csv') 
df.info()
df.iloc[160, 5]

<class 'pandas.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        250 non-null    int64  
 1   id                250 non-null    int64  
 2   domain            250 non-null    str    
 3   type              238 non-null    str    
 4   url               250 non-null    str    
 5   content           250 non-null    str    
 6   scraped_at        250 non-null    str    
 7   inserted_at       250 non-null    str    
 8   updated_at        250 non-null    str    
 9   title             250 non-null    str    
 10  authors           170 non-null    str    
 11  keywords          0 non-null      float64
 12  meta_keywords     250 non-null    str    
 13  meta_description  54 non-null     str    
 14  tags              27 non-null     str    
 15  summary           0 non-null      float64
dtypes: float64(2), int64(2), str(12)
memory usage: 31.4 KB


'Worldwide Laser Technology Market Analysis Report 2020 – Challenges & Opportunities\n\nHeadline: Bitcoin & Blockchain Searches Exceed Trump! Blockchain Stocks Are Next!\n\nNew Market Research Report on “Laser Technology Market by Type (Solid – YAG laser, Fiber laser, Thin Disk Laser, Liquid, Gas – Argon Ion Laser, Excimer, CO2& Others), Application (Medical, Industrial, Military, Research, Consumer, & Others), and Geography – Trends & Forecast to 2013 – 2020adds to ReportsnReports.com.\n\nLASER (Light Amplification by Simulated Emission of Radiation) technology has been used forvarious applications such as medical, industrial, military, research, commercial, consumer, automotive, and communication. The purpose of these applications is either scientific/laboratory experiments or the measurement/exploration of distances. Lasers have become robust, compact, and versatile with technological developments. These developments have triggered the commercial and industrial applications.\n\nThis

In [ ]:
# Most of these expression was taken from stack overflow
def clean_text(series):
    series = series.str.lower()
    series = series.str.replace(r'https?://\S+|www\.\S+', '<URL>', regex=True)
    series = series.str.replace(r'[\w.]+@[\w]+\.[\w]+', '<EMAIL>' ,regex=True)
    series = series.str.replace(r'\d{4}-\d{2}-\d{2}', '<DATE>', regex=True)
    series = series.str.replace(r'\d{1,2}/\d{1,2}/\d{2,4}', '<DATE>', regex=True)
    series = series.str.replace(r'\d+', '<NUM>', regex=True)    
    series = series.str.replace(r'\s+', ' ', regex=True)
    series = series.str.strip()
    return series

df['clean_txt'] = clean_text(df['content'])


'worldwide laser technology market analysis report <NUM> – challenges & opportunities headline: bitcoin & blockchain searches exceed trump! blockchain stocks are next! new market research report on “laser technology market by type (solid – yag laser, fiber laser, thin disk laser, liquid, gas – argon ion laser, excimer, co<NUM>& others), application (medical, industrial, military, research, consumer, & others), and geography – trends & forecast to <NUM> – <NUM>adds to reportsnreports.com. laser (light amplification by simulated emission of radiation) technology has been used forvarious applications such as medical, industrial, military, research, commercial, consumer, automotive, and communication. the purpose of these applications is either scientific/laboratory experiments or the measurement/exploration of distances. lasers have become robust, compact, and versatile with technological developments. these developments have triggered the commercial and industrial applications. this lase

In [ ]:
#Replacing number with <NUM>



0      sometimes the power of christmas will make you...
1      awakening of <NUM> strands of dna – “reconnect...
2      never hike alone: a friday the <NUM>th fan fil...
3      when a rare shark was caught, scientists were ...
4      donald trump has the unnerving ability to abil...
                             ...                        
245    prison for rahm, god’s work and many others\n\...
246    <NUM> useful items for your tiny home\n\nheadl...
247    former cia director michael hayden said thursd...
248    antonio sabato jr. says hollywood's liberal el...
249    former u.s. president bill clinton on monday c...
Name: clean_txt, Length: 250, dtype: str

0      sometimes the power of christmas will make you...
1      awakening of <NUM> strands of dna – “reconnect...
2      never hike alone: a friday the <NUM>th fan fil...
3      when a rare shark was caught, scientists were ...
4      donald trump has the unnerving ability to abil...
                             ...                        
245    prison for rahm, god’s work and many others he...
246    <NUM> useful items for your tiny home headline...
247    former cia director michael hayden said thursd...
248    antonio sabato jr. says hollywood's liberal el...
249    former u.s. president bill clinton on monday c...
Name: clean_txt, Length: 250, dtype: str

In [ ]:
# replacing emails with <EMAIL>
df['clean_txt']


0      sometimes the power of christmas will make you...
1      awakening of <NUM> strands of dna – “reconnect...
2      never hike alone: a friday the <NUM>th fan fil...
3      when a rare shark was caught, scientists were ...
4      donald trump has the unnerving ability to abil...
                             ...                        
245    prison for rahm, god’s work and many others he...
246    <NUM> useful items for your tiny home headline...
247    former cia director michael hayden said thursd...
248    antonio sabato jr. says hollywood's liberal el...
249    former u.s. president bill clinton on monday c...
Name: clean_txt, Length: 250, dtype: str

In [148]:
df['clean_txt'] = df['clean_txt'].str.replace(r'https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)', '<URL>' ,regex=True)

df[df['clean_txt'].str.contains('<URL>')]
df[df['clean_txt'].str.contains('<EMAIL>')]

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,clean_txt,matches
160,160,24998,beforeitsnews.com,fake,http://beforeitsnews.com/business/2014/12/worl...,Worldwide Laser Technology Market Analysis Rep...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Worldwide Laser Technology Market Analysis Rep...,Evan Wyatt,NaN,[''],NaN,NaN,NaN,worldwide laser technology market analysis rep...,[]
173,173,26829,beforeitsnews.com,fake,http://beforeitsnews.com/survival/2013/11/cowb...,Cowboy Mounted Shooting Association Joins Nort...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cowboy Mounted Shooting Association Joins Nort...,NaN,NaN,[''],NaN,NaN,NaN,cowboy mounted shooting association joins nort...,[]
175,175,27128,beforeitsnews.com,fake,http://beforeitsnews.com/market-research-repor...,Trends Global Connected Enterprise Market 2017...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Trends Global Connected Enterprise Market 2017...,Data Bridge Market Research,NaN,[''],NaN,NaN,NaN,trends global connected enterprise market <NUM...,[]
210,210,34422,beforeitsnews.com,fake,http://beforeitsnews.com/agriculture/2017/11/f...,Feed Amino Acids Market – Competitive Landscap...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Feed Amino Acids Market – Competitive Landscap...,NaN,NaN,[''],NaN,NaN,NaN,feed amino acids market – competitive landscap...,[]
233,233,37647,beforeitsnews.com,fake,http://beforeitsnews.com/opinion-liberal/2018/...,What is Behind New MSM Push for Invasion of Sy...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,What is Behind New MSM Push for Invasion of Sy...,American Everyman,NaN,[''],NaN,NaN,NaN,what is behind new msm push for invasion of sy...,[]


In [153]:
df.iloc[160, 16]

'worldwide laser technology market analysis report <NUM> – challenges & opportunities headline: bitcoin & blockchain searches exceed trump! blockchain stocks are next! new market research report on “laser technology market by type (solid – yag laser, fiber laser, thin disk laser, liquid, gas – argon ion laser, excimer, co<NUM>& others), application (medical, industrial, military, research, consumer, & others), and geography – trends & forecast to <NUM> – <NUM>adds to reportsnreports.com. laser (light amplification by simulated emission of radiation) technology has been used forvarious applications such as medical, industrial, military, research, commercial, consumer, automotive, and communication. the purpose of these applications is either scientific/laboratory experiments or the measurement/exploration of distances. lasers have become robust, compact, and versatile with technological developments. these developments have triggered the commercial and industrial applications. this lase